In [1]:
# Load packages
suppressPackageStartupMessages({
  library(DESeq2)
  library(tidyverse)
  library(ggplot2)
  library(pheatmap)
})

# Load region lengths
region_lengths <- read_tsv("master/union_500bp_nonoverlapping_with_ids.bed", col_names = FALSE) %>%
  setNames(c("chr", "start", "end", "region_id")) %>%
  mutate(length = end - start) %>%
  select(region_id, length)

# === Step 1: Load data ===
load_species <- function(file, species_name, region_lengths) {
  df <- read_tsv(file) %>%
    select(-region) %>%
    left_join(region_lengths, by = "region_id")

  df_counts <- df %>%
    mutate(across(-c(region_id, length), ~ round(. * length))) %>%
    select(-length) %>%
    rename_with(~ paste0(species_name, "_", .), -region_id)

  return(df_counts)
}


bonobo   <- load_species("Bonobo_quantification.tsv", "Bonobo", region_lengths)
human    <- load_species("Human_quantification.tsv", "Human", region_lengths)
macaque  <- load_species("Macaque_quantification.tsv", "Macaque", region_lengths)
marmoset <- load_species("Marmoset_quantification.tsv", "Marmoset", region_lengths)
chimp <- load_species("Chimpanzee_quantification.tsv", "Chimpanzee", region_lengths)
gorilla <- load_species("Gorilla_quantification.tsv", "Gorilla", region_lengths)



Rows: 1003109 Columns: 4
-- Column specification ----------------------------------------------------------------------------------------------------------
Delimiter: "\t"
chr (2): X1, X4
dbl (2): X2, X3

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 955375 Columns: 37
-- Column specification ----------------------------------------------------------------------------------------------------------
Delimiter: "\t"
chr  (2): region_id, region
dbl (35): Adipocytes, BEST4+_cells, Colonocytes, Crypt_Fibroblasts_WNT2B+, E...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1003109 Columns: 34
-- Column specification ----------------------------------------------------------------------------------------------------------
Delimiter: "\t"
chr  (2): region_id, region
dbl (

In [2]:
summarize_species_counts <- function(df, species_name) {
  df_long <- df %>%
    pivot_longer(-region_id, names_to = "sample", values_to = "count") %>%
    mutate(
      celltype = sub(paste0(species_name, "_"), "", sample),
      species = species_name
    )
  
  df_summary <- df_long %>%
    group_by(species, celltype) %>%
    summarize(
      total_signal = sum(count),
      mean_per_region = mean(count),
      median_per_region = median(count),
      .groups = "drop"
    )
  
  return(df_summary)
}

# Apply to each species
summaries <- list(
  summarize_species_counts(bonobo, "Bonobo"),
  summarize_species_counts(human, "Human"),
  summarize_species_counts(macaque, "Macaque"),
  summarize_species_counts(marmoset, "Marmoset"),
  summarize_species_counts(chimp, "Chimpanzee"),
  summarize_species_counts(gorilla, "Gorilla")
)

combined_summary <- bind_rows(summaries)

In [3]:
combined_summary

species,celltype,total_signal,mean_per_region,median_per_region
<chr>,<chr>,<dbl>,<dbl>,<dbl>
Bonobo,Adipocytes,227143350,237.75308,148
Bonobo,BEST4+_cells,506696723,530.36423,0
Bonobo,Colonocytes,145401369,152.19298,66
Bonobo,Crypt_Fibroblasts_WNT2B+,113088296,118.37058,62
Bonobo,ECs,111171379,116.36413,63
Bonobo,EECs,214026620,224.02368,124
Bonobo,Enteric_glia,277589138,290.55516,195
Bonobo,Enteric_neurons,973510459,1018.98256,0
Bonobo,Enterocytes,72869164,76.27284,34


In [ ]:
# === Step 2: Combine all into one matrix ===
all_data <- list(bonobo, human, macaque, marmoset,chimp,gorilla) %>%
  reduce(inner_join, by = "region_id") %>%
  column_to_rownames("region_id")

# Fill NAs with 0
all_data[is.na(all_data)] <- 0



In [2]:
dim(all_data)

[1] 63226   197

In [ ]:
# === Step 4: Create colData ===
sample_names <- colnames(all_data)
coldata <- data.frame(
  sample = sample_names,
  species = sub("_.*", "", sample_names),
  celltype = sub(".*_", "", sample_names)
)
rownames(coldata) <- sample_names

# === Step 5: Build DESeq2 object ===
dds <- DESeqDataSetFromMatrix(
  countData = round(all_data),  # must be integers
  colData = coldata,
  design = ~ species + celltype
)

# === Step 6: Run DESeq2 and PCA ===
dds <- DESeq(dds)
vsd <- vst(dds)

# PCA plot
pcaData <- plotPCA(vsd, intgroup = c("species", "celltype"), returnData = TRUE)
ggplot(pcaData, aes(PC1, PC2, color = species, shape = celltype)) +
  geom_point(size = 3) +
  theme_minimal() +
  labs(title = "PCA of accessibility signal (shared consensus regions)")


converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating size factors

  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]

estimating dispersions

gene-wise dispersion estimates



In [30]:
library(DESeq2)
library(tidyverse)
library(ggplot2)
library(pheatmap)

# === Create output dir ===
out_dir <- "species_deseq2_results"
dir.create(out_dir, showWarnings = FALSE)

# === Load BED coordinates with region IDs ===
bed_coords <- read_tsv("master/shared_consensus_with_ids.hg38.bed",
                       col_names = c("chr", "start", "end", "region_id"))

# === Species to compare against Human ===
species_to_compare <- c("Chimpanzee", "Bonobo", "Gorilla", "Macaque", "Marmoset")

# === Loop over species ===
for (sp in species_to_compare) {
  message("🧬 Running DESeq2: Human vs ", sp)

  # Subset and prepare
  dds_subset <- dds[, dds$species %in% c("Human", sp)]
  dds_subset$species <- droplevels(dds_subset$species)
  design(dds_subset) <- ~ species

  # Run DESeq2
  dds_subset <- DESeq(dds_subset, quiet = TRUE)

  # Get results
  res <- results(dds_subset, contrast = c("species", "Human", sp)) %>%
    as.data.frame() %>%
    rownames_to_column("region_id")

  # Save full result table
  write_tsv(res, file.path(out_dir, paste0("DESeq2_Human_vs_", sp, "_AllCelltypes.tsv")))

  # Filter significant (padj < 0.05)
  sig_peaks <- res %>%
    filter(!is.na(padj) & padj < 0.05)

  # Join with coordinates
  sig_with_coords <- left_join(sig_peaks, bed_coords, by = "region_id")

  # === BASIC BEDs ===
  human_basic <- sig_with_coords %>%
    filter(log2FoldChange > 0) %>%
    select(chr, start, end)
  sp_basic <- sig_with_coords %>%
    filter(log2FoldChange < 0) %>%
    select(chr, start, end)

  write_tsv(human_basic, file.path(out_dir, paste0("DESeq2_Human_vs_", sp, "_HumanSpecific.bed")), col_names = FALSE)
  write_tsv(sp_basic, file.path(out_dir, paste0("DESeq2_Human_vs_", sp, "_", sp, "Specific.bed")), col_names = FALSE)

  # === STRONGER log2FC FILTERED BEDs (|log2FC| > 1) ===
  human_strong <- sig_with_coords %>%
    filter(log2FoldChange > 1) %>%
    select(chr, start, end)
  sp_strong <- sig_with_coords %>%
    filter(log2FoldChange < -1) %>%
    select(chr, start, end)

  write_tsv(human_strong, file.path(out_dir, paste0("DESeq2_Human_vs_", sp, "_HumanSpecific_logFC1.bed")), col_names = FALSE)
  write_tsv(sp_strong, file.path(out_dir, paste0("DESeq2_Human_vs_", sp, "_", sp, "Specific_logFC1.bed")), col_names = FALSE)

  message("✅ Done: ", sp, " | Sig: ", nrow(sig_peaks), 
          " | Human> ", nrow(human_basic), " (", nrow(human_strong), " >1)",
          " | ", sp, "> ", nrow(sp_basic), " (", nrow(sp_strong), " >1)")
}

Rows: 368636
Columns: 4
-- Column specifi
Delimiter: "\t"
chr (2): chr, region_id
dbl (2): start, end

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
<U+0001F9EC> Running DESeq2: Human vs Chimpanzee

<U+2705> Done: Chimpanzee | Sig: 914 | Human> 781 (774 >1) | Chimpanzee> 133 (132 >1)

<U+0001F9EC> Running DESeq2: Human vs Bonobo

<U+2705> Done: Bonobo | Sig: 15796 | Human> 230 (176 >1) | Bonobo> 15566 (15534 >1)

<U+0001F9EC> Running DESeq2: Human vs Gorilla

<U+2705> Done: Gorilla | Sig: 1711 | Human> 1167 (1146 >1) | Gorilla> 544 (539 >1)

<U+0001F9EC> Running DESeq2: Human vs Macaque

<U+2705> Done: Macaque | Sig: 456 | Human> 377 (377 >1) | Macaque> 79 (79 >1)

<U+0001F9EC> Running DESeq2: Human vs Marmoset

<U+2705> Done: Marmoset | Sig: 2412 | Human> 1452 (1423 >1) | Marmoset> 960 (940 >1)



In [41]:
library(ggplot2)
library(gtools)
library(readr)
library(dplyr)
library(ggrepel)
library(tidyverse)

plot_manhattan <- function(deseq_file, bed_file, chromsizes_file, output_file, plot_title = NULL, width = 16, height = 5) {
  # Load data
  deseq_res <- read_tsv(deseq_file, show_col_types = FALSE)
  bed_coords <- read_tsv(bed_file, col_names = c("chr", "start", "end", "region_id"), show_col_types = FALSE)
  chromsizes <- read_tsv(chromsizes_file, col_names = c("chr", "size"), show_col_types = FALSE)

  # Clean and order chromosomes
  chromsizes <- chromsizes %>%
    filter(chr %in% bed_coords$chr) %>%
    arrange(mixedorder(chr)) %>%
    mutate(offset = lag(cumsum(size), default = 0))  # Genomic offset for each chromosome

  # Join DESeq2 + BED
  plot_data <- left_join(deseq_res, bed_coords, by = "region_id") %>%
    filter(!is.na(chr)) %>%
    left_join(chromsizes, by = "chr") %>%
    mutate(genomic_pos = offset + (start + end) / 2)

  # X-axis labels at chromosome midpoints
  chr_ticks <- chromsizes %>%
    mutate(midpoint = offset + size / 2)

  # Default title
  if (is.null(plot_title)) {
    plot_title <- tools::file_path_sans_ext(basename(deseq_file))
  }

  # Plot
  p <- ggplot(plot_data, aes(x = genomic_pos, y = -log10(pvalue), color = chr)) +
    geom_point(size = 0.5, alpha = 0.8, show.legend = FALSE) +
    scale_color_manual(values = rep(c("#1f77b4", "#ff7f0e"), length.out = nrow(chr_ticks))) +
    scale_x_continuous(
      label = chr_ticks$chr,
      breaks = chr_ticks$midpoint,
      expand = expansion(mult = c(0.01, 0.01))
    ) +
    labs(
      x = "Chromosome",
      y = expression(-log[10](pvalue)),
      title = plot_title
    ) +
    theme_minimal(base_size = 14) +
    theme(
      axis.text.x = element_text(angle = 90, vjust = 0.5, size = 10),
      panel.grid.major.x = element_blank(),
      plot.title = element_text(hjust = 0.5)
    )

  ggsave(output_file, p, width = width, height = height)
  message("✅ Saved plot to ", output_file)
  return(p)
}

In [43]:
dir.create("plots", showWarnings = FALSE)

comparisons <- c("Human_vs_Bonobo", "Human_vs_Chimpanzee", "Human_vs_Gorilla", "Human_vs_Macaque", "Human_vs_Marmoset")

for (comp in comparisons) {
  deseq_file <- paste0("species_deseq2_results/DESeq2_", comp, "_AllCelltypes.tsv")
  output_file <- paste0("plots/Manhattan_", comp, ".pdf")



    plot_manhattan(
      deseq_file = deseq_file,
      bed_file = "master/shared_consensus_with_ids.hg38.bed",
      chromsizes_file = "~/jjans/analysis/cerebellum/genomes_new/homo_sapiens/hg38.chrom.sizes",
      output_file = output_file
    )

}

Warning message:
"Removed 74 rows containing missing values (geom_point)."
<U+2705> Saved plot to plots/Manhattan_Human_vs_Bonobo.pdf

Warning message:
"Removed 76 rows containing missing values (geom_point)."
<U+2705> Saved plot to plots/Manhattan_Human_vs_Chimpanzee.pdf

Warning message:
"Removed 69 rows containing missing values (geom_point)."
<U+2705> Saved plot to plots/Manhattan_Human_vs_Gorilla.pdf

Warning message:
"Removed 62 rows containing missing values (geom_point)."
<U+2705> Saved plot to plots/Manhattan_Human_vs_Macaque.pdf

Warning message:
"Removed 86 rows containing missing values (geom_point)."
<U+2705> Saved plot to plots/Manhattan_Human_vs_Marmoset.pdf



In [44]:
library(limma)

# Subset Enterocytes only
enterocyte_counts <- all_data[, endsWith(colnames(all_data), "_Enterocytes")]
enterocyte_counts <- enterocyte_counts[, c("Human_Enterocytes", "Bonobo_Enterocytes")]

# Design
group <- factor(c("Human", "Bonobo"))
design <- model.matrix(~group)

# voom normalization
v <- voom(enterocyte_counts, design, plot = TRUE)

# Fit linear model
fit <- lmFit(v, design)
fit <- eBayes(fit)

# Results
topTable(fit, coef = 2, n = Inf)


Attaching package: 'limma'


The following object is masked from 'package:DESeq2':

    plotMA


The following object is masked from 'package:BiocGenerics':

    plotMA


Warning message in voom(enterocyte_counts, design, plot = TRUE):
"The experimental design has no replication. Setting weights to 1."


ERROR: Error in .ebayes(fit = fit, proportion = proportion, stdev.coef.lim = stdev.coef.lim, : No residual degrees of freedom in linear model fits
